What does tf-idf mean? 

Tf-idf stands for term frequency-inverse document frequency, and the tf-idf weight is a weight often used in information retrieval and text mining. This weight is a statistical measure used to evaluate how important a word is to a document in a collection or corpus. The importance increases proportionally to the number of times a word appears in the document but is offset by the frequency of the word in the corpus. Variations of the tf-idf weighting scheme are often used by search engines as a central tool in scoring and ranking a document's relevance given a user query.

One of the simplest ranking functions is computed by summing the tf-idf for each query term; many more sophisticated ranking functions are variants of this simple model.

Tf-idf can be successfully used for stop-words filtering in various subject fields including text summarization and classification.


How to Compute:

Typically, the tf-idf weight is composed by two terms: the first computes the normalized Term Frequency (TF), aka. the number of times a word appears in a document, divided by the total number of words in that document; the second term is the Inverse Document Frequency (IDF), computed as the logarithm of the number of the documents in the corpus divided by the number of documents where the specific term appears.

TF: Term Frequency, which measures how frequently a term occurs in a document. Since every document is different in length, it is possible that a term would appear much more times in long documents than shorter ones. Thus, the term frequency is often divided by the document length (aka. the total number of terms in the document) as a way of normalization.

IDF: Inverse Document Frequency, which measures how important a term is. While computing TF, all terms are considered equally important. However it is known that certain terms, such as "is", "of", and "that", may appear a lot of times but have little importance. Thus we need to weight down the frequent terms while scale up the rare ones.

Example
Consider a document containing 100 words wherein the word cat appears 3 times. The term frequency (i.e., tf) for cat is then (3 / 100) = 0.03. Now, assume we have 10 million documents and the word cat appears in one thousand of these. Then, the inverse document frequency (i.e., idf) is calculated as log(10,000,000 / 1,000) = 4. Thus, the Tf-idf weight is the product of these quantities: 0.03 * 4 = 0.12.


In [1]:
corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

Custom Implementation of tf-idf: 

In [2]:
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy


def fit(corpus):
    unique_words=set()    #set of unique words in corpus
    for sentence in tqdm(corpus):
        for word in sentence.split(' '):
            if len(word)>=2:
                unique_words.add(word)
    unique_words=sorted(list(unique_words))   #sorted list of unique words
    dict_vocab={j:i for i,j in enumerate(unique_words)}   #Creating Dict from unique words
    return unique_words,dict_vocab

vocab,dict_vocab=fit(corpus)
print(vocab)
#print(dict_vocab)

100%|██████████| 4/4 [00:00<00:00, 4949.03it/s]

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [3]:
def transform(corpus):
    row=[]       #row number for sparse matrix
    col=[]       #coloumn number for sparse matrix
    values=[]    #corrosponding value for sparse matrix
    
    idf=[0]*len(vocab)     #idf value for each word in vocab
    
    for i,sentence in enumerate(tqdm(corpus)):
        word_freq=dict(Counter(sentence.split(' ')))
        word_freq=dict(sorted(word_freq.items()))
        l=len(sentence.split(' '))
        for word, freq in word_freq.items():
            if len(word)>=2:
                if dict_vocab.get(word,-1)!=-1:
                    row+=i,
                    col+=dict_vocab[word],
                    values+=freq/l,
                    idf[dict_vocab[word]]+=1
    
    for i in range(len(idf)):
        idf[i]=1+math.log((len(corpus)+1)/(1+idf[i]))
    
    for i in range(len(values)):
        p=dict_vocab[vocab[col[i]]]
        values[i]=values[i]*idf[p]      #tf-idf=tf*idf

    output=csr_matrix((values,(row,col)),shape=(len(corpus),len(vocab)))   #sparse matrix formation
    return output,idf

feature,idf=transform(corpus)
feature=normalize(feature,norm='l2',axis=1)     #l2 norm calculation
print(idf)

100%|██████████| 4/4 [00:00<00:00, 4895.60it/s]

[1.916290731874155, 1.2231435513142097, 1.5108256237659907, 1.0, 1.916290731874155, 1.916290731874155, 1.0, 1.916290731874155, 1.0]


In [4]:
print(feature[0])

  (0, 1)	0.4697913855799205
  (0, 2)	0.580285823684436
  (0, 3)	0.3840852409148149
  (0, 6)	0.3840852409148149
  (0, 8)	0.3840852409148149


In [5]:
print(feature[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


SkLearn Implementation for verification:

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
vectorizer.fit(corpus)
skl_output = vectorizer.transform(corpus)

In [7]:
print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [8]:
print(vectorizer.idf_)

[1.91629073 1.22314355 1.51082562 1.         1.91629073 1.91629073
 1.         1.91629073 1.        ]


In [9]:
skl_output.shape

(4, 9)

In [10]:
print(skl_output[0])

  (0, 8)	0.38408524091481483
  (0, 6)	0.38408524091481483
  (0, 3)	0.38408524091481483
  (0, 2)	0.5802858236844359
  (0, 1)	0.46979138557992045


In [11]:
print(skl_output[0].toarray())

[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]
